# Risk Analysis and Profit Optimization for Oil Well Development in Multiple Regions

Our client is the company "RosGosOil".  We need to determine where to drill a new oil well.

You have been provided with oil samples from three regions: in each region, there are 10,000 oil fields where the quality of the oil and the volume of its reserves have been measured. Build a machine learning model that will help determine the region where extraction will bring the highest profit. Analyze the potential profit and risks using the Bootstrap technique.

Steps for selecting a location:

- In the selected region, search for oil fields and determine the feature values for each field.
- Build a model and estimate the volume of reserves.
- Select the fields with the highest estimated values. The number of fields depends on the company's budget and the - cost of developing one well.
- Profit is equal to the total profit from the selected oil fields.

## Data preprocessing

Import of necessary libraries

In [1]:
import pandas as pd
import numpy as np
from scipy import stats as st 

from sklearn.model_selection import train_test_split

import seaborn as sns

from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression 

from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score

from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import make_pipeline
from sklearn.datasets import make_classification
pd.options.mode.chained_assignment = None

state = np.random.RandomState(2020)



Let's save the datases in separate pandas dataframes: 

In [2]:
data0 = pd.read_csv('geo_data_0.csv')  
data1 = pd.read_csv('geo_data_1.csv')
data2 = pd.read_csv('geo_data_2.csv')

In [3]:
print(data0.info(),data1.info(),data2.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null 

Check for the duplicates:

In [4]:
data0[data0.duplicated(subset = ['id'])]

,id,f0,f1,f2,product
7530,HZww2,1.061194,-0.373969,10.430210,158.828695
41724,bxg6G,-0.823752,0.546319,3.630479,93.007798
51970,A5aEY,-0.180335,0.935548,-2.094773,33.020205
63593,QcMuo,0.635635,-0.473422,0.862670,64.578675
66136,74z30,1.084962,-0.312358,6.990771,127.643327
69163,AGS9W,-0.933795,0.116194,-3.655896,19.230453
75715,Tdehs,0.112079,0.430296,3.218993,60.964018
90815,fiKDv,0.049883,0.841313,6.394613,137.346586
92341,TtcGQ,0.110711,1.022689,0.911381,101.318008
97785,bsk9y,0.378429,0.005837,0.160827,160.637302


In [5]:
data1[data1.duplicated(subset = ['id'])]

,id,f0,f1,f2,product
41906,LHZR0,-8.989672,-4.286607,2.009139,57.085625
82178,bfPNe,-6.202799,-4.820045,2.995107,84.038886
82873,wt4Uk,10.259972,-9.376355,4.994297,134.766305
84461,5ltQ6,18.213839,2.191999,3.993869,107.813044


In [6]:
data2[data2.duplicated(subset = ['id'])]

,id,f0,f1,f2,product
43233,xCHr8,-0.847066,2.101796,5.597130,184.388641
49564,VF7Jo,-0.883115,0.560537,0.723601,136.233420
55967,KUPhW,1.211150,3.176408,5.543540,132.831802
95090,Vcm5J,2.587702,1.986875,2.482245,92.327572


There are very few duplicates, so they will not affect the model. Let's remove the 'id' column from each dataset: this feature will not help train the model.

In [7]:
data0=data0.drop(['id'], axis=1)
data0.head()

,f0,f1,f2,product
0,0.705745,-0.497823,1.221170,105.280062
1,1.334711,-0.340164,4.365080,73.037750
2,1.022732,0.151990,1.419926,85.265647
3,-0.032172,0.139033,2.978566,168.620776
4,1.988431,0.155413,4.751769,154.036647


In [8]:
data1=data1.drop(['id'], axis=1)
data1.head()

,f0,f1,f2,product
0,-15.001348,-8.276000,-0.005876,3.179103
1,14.272088,-3.475083,0.999183,26.953261
2,6.263187,-5.948386,5.001160,134.766305
3,-13.081196,-11.506057,4.999415,137.945408
4,12.702195,-8.147433,5.004363,134.766305


In [9]:
data2=data2.drop(['id'], axis=1)
data2.head()

,f0,f1,f2,product
0,-1.146987,0.963328,-0.828965,27.758673
1,0.262778,0.269839,-2.530187,56.069697
2,0.194587,0.289035,-5.586433,62.871910
3,2.236060,-0.553760,0.930038,114.572842
4,-0.515993,1.716266,5.899011,149.600746


We will split the data into training and validation sets and extract the target variable. Since we have three datasets, for convenience, we will create separate functions for each action.

In [10]:
def split_data(data):  
    data_train, data_valid = train_test_split(data, test_size=0.25, random_state=12345, shuffle = True)
    return data_train, data_valid

# and a function for 'features' and 'target'
def get_features(data, field):
    return data.drop([field], axis = 1)
def get_target(data, field):
    return data[field]

def ftr_trgt_split(data, target_field):
    features_train = get_features(data, target_field)
    target_train = get_target(data, target_field)
    return features_train, target_train

And now apply the functions to the dataframe

In [11]:
data0_train, data0_valid = split_data(data0)

features0_train, target0_train = ftr_trgt_split(data0_train, 'product')
features0_valid, target0_valid = ftr_trgt_split(data0_valid, 'product')

In [12]:
data1_train, data1_valid = split_data(data1)

features1_train, target1_train = ftr_trgt_split(data1_train, 'product')
features1_valid, target1_valid = ftr_trgt_split(data1_valid, 'product')

In [13]:
data2_train, data2_valid = split_data(data2)

features2_train, target2_train = ftr_trgt_split(data2_train, 'product')
features2_valid, target2_valid = ftr_trgt_split(data2_valid, 'product')

Now it's time to scale the data, will create a separate function

In [14]:
def df_scaler(data_train, data_valid):
    numeric = data_train.columns

    scaler = StandardScaler()
    scaler.fit(data_train[numeric])
    data_train[numeric] = scaler.transform(data_train[numeric])
    data_valid[numeric] = scaler.transform(data_valid[numeric])
    return data_train, data_valid

In [15]:
features0_train, features0_valid = df_scaler(features0_train, features0_valid)
features1_train, features1_valid = df_scaler(features1_train, features1_valid)
features2_train, features2_valid = df_scaler(features2_train, features2_valid)

So, we have loaded and examined the data, ensuring that there are no missing values. Some duplicates were found in the `id` column. We decided to remove this feature as the unique objects would not help the model learn.

Next, we divided the data for each region into training and validation sets in a 75:25 ratio and extracted the target variable.

Finally, we performed feature scaling.

## Model Training and Evaluation
According to the project's specifications, we are only able to work with linear regression for the given data.

Let's create functions for model training, prediction, and obtaining statistical metrics. We will organize the results in a final table.

In [16]:
def model_predict(features_train, target_train, features_valid, target_valid):
    scaler = StandardScaler()

    scaler.fit(features_train)

    features_train = scaler.transform(features_train)
    features_valid = scaler.transform(features_valid)

    model = LinearRegression()
    model.fit(features_train, target_train)

    predicted_valid = model.predict(features_valid)
    return model.score(features_valid, target_valid), predicted_valid

Region 0

In [17]:
model_0_score, predicted_valid_0 = model_predict(features_train=features0_train,
                                                 target_train=target0_train,
                                                 features_valid=features0_valid,
                                                 target_valid=target0_valid)

Region 1

In [18]:
model_1_score, predicted_valid_1 = model_predict(features_train=features1_train,
                                                 target_train=target1_train,
                                                 features_valid=features1_valid,
                                                 target_valid=target1_valid)

region 2

In [19]:
model_2_score, predicted_valid_2 = model_predict(features_train=features2_train,
                                                 target_train=target2_train,
                                                 features_valid=features2_valid,
                                                 target_valid=target2_valid)

Separate tab;e for the results

In [20]:
target0_predict = pd.concat([target0_valid.reset_index(drop=True), pd.DataFrame(predicted_valid_0)], axis=1)
target1_predict = pd.concat([target1_valid.reset_index(drop=True), pd.DataFrame(predicted_valid_1)], axis=1)
target2_predict = pd.concat([target2_valid.reset_index(drop=True), pd.DataFrame(predicted_valid_2)], axis=1)

target0_predict = target0_predict.rename(columns={0:'predicted_product'})
target1_predict = target1_predict.rename(columns={0:'predicted_product'})
target2_predict = target2_predict.rename(columns={0:'predicted_product'})

In [21]:
target1_valid.sum()


1718078.4006089992

In [22]:
model_results = pd.DataFrame({'Real Reserves of Resources'        :[target0_valid.sum(),
                                                               target1_valid.sum(),
                                                               target2_valid.sum()],
                              'Predicted Reserves of Resources'      : [predicted_valid_0.sum(),
                                                                predicted_valid_1.sum(),
                                                                predicted_valid_2.sum()],
                              'Avg Predicted Reserves of Resources' : [predicted_valid_0.mean(),
                                                                predicted_valid_1.mean(),
                                                                predicted_valid_2.mean()],
                              'RMSE'                         : [(mean_squared_error(target0_valid, predicted_valid_0))**0.5,
                                                                (mean_squared_error(target1_valid, predicted_valid_1))**0.5,
                                                                (mean_squared_error(target2_valid, predicted_valid_2))**0.5],
                              'R2-score'                     : [model_0_score,
                                                                model_1_score,
                                                                model_2_score]
}
)

model_results


,Real Reserves of Resources,Predicted Reserves of Resources,Avg Predicted Reserves of Resources,RMSE,R2-score
0,2.301965e+06,2.314814e+06,92.592568,37.579422,0.279943
1,1.718078e+06,1.718214e+06,68.728547,0.893099,0.999623
2,2.372106e+06,2.374126e+06,94.965046,40.029709,0.205248


We prepared the linear regression model, trained it, and obtained the main statistical metrics. Let's proceed with the calculation of profit and risks according to the given conditions. The minimum value of RMSE was obtained in Region 1.

## Profit Calculation Preparation
During the exploration of a region, 500 points are investigated, from which the top 200 are selected for development using machine learning.

The budget for well development in the province is 10 billion rubles.
At current prices, one barrel of resources generates a revenue of 450 rubles.
The revenue per unit of product is 450,000 rubles, as the volume is given in thousands of barrels.

Let's create new variables:

In [23]:
BUDGET = 10000000000
PRICE = 450000
OIL_SELECTED = 200
VALUE_TRESHOLD = BUDGET/PRICE/OIL_SELECTED 
PROVINCE_TRESHOLD=BUDGET/PRICE
print('Break-even volume of resources for developing a new well: ', VALUE_TRESHOLD)
print('Break-even volume of resources for developing a new province: ', PROVINCE_TRESHOLD)


Break-even volume of resources for developing a new well:  111.11111111111111
Break-even volume of resources for developing a new province:  22222.222222222223


Thus, for a break-even development of a well, the volume of resources should be at least 112 units, while for the region it should be at least 2,223 units.

## Profit and risk calculation

Let's create a function for calculating profits:

In [24]:
def material_sum_revenue(target, probabilities, count):
    probs_sorted = probabilities.sort_values(ascending=False)
    selected = target[probs_sorted.index][:count]
    material_sum = selected.sum()
    material_revenue = material_sum*PRICE - BUDGET
    return pd.DataFrame({'material_sum':[material_sum], 'material_revenue':[material_revenue]})

Create a bootstrap calculation function: 

In [25]:
def bootstrap(target_valid, probabilities_valid, hole_count):
    region_boot = pd.DataFrame()
    for i in range(1000):
        target_subsample = target_valid.sample(n=hole_count, replace=True, random_state=state)
        probs_subsample = probabilities_valid[target_subsample.index]
        region_boot = region_boot.append(material_sum_revenue(target_subsample, probs_subsample, 200), ignore_index=True)
    return  region_boot

Let's proceed with the distribution of resource reserves and profits for the three provinces using the bootstrap function.

In [26]:
region_0_boot = bootstrap(target_valid=target0_predict['product'],
                          probabilities_valid=target0_predict['predicted_product'],
                          hole_count=500)

region_1_boot = bootstrap(target_valid=target1_predict['product'],
                            probabilities_valid=target1_predict['predicted_product'],
                            hole_count=500)

region_2_boot = bootstrap(target_valid=target2_predict['product'],
                            probabilities_valid=target2_predict['predicted_product'],
                            hole_count=500)

/var/folders/cy/1f9vz20x39s2ctm4tynllrt00000gn/T/ipykernel_56668/1492736127.py:6: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  region_boot = region_boot.append(material_sum_revenue(target_subsample, probs_subsample, 200), ignore_index=True)
/var/folders/cy/1f9vz20x39s2ctm4tynllrt00000gn/T/ipykernel_56668/1492736127.py:6: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  region_boot = region_boot.append(material_sum_revenue(target_subsample, probs_subsample, 200), ignore_index=True)
/var/folders/cy/1f9vz20x39s2ctm4tynllrt00000gn/T/ipykernel_56668/1492736127.py:6: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  region_boot = region_boot.append(material_sum_revenue(target_subsample, probs_subsample, 200), ignore_index=True)
/var/fo

/var/folders/cy/1f9vz20x39s2ctm4tynllrt00000gn/T/ipykernel_56668/1492736127.py:6: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  region_boot = region_boot.append(material_sum_revenue(target_subsample, probs_subsample, 200), ignore_index=True)
/var/folders/cy/1f9vz20x39s2ctm4tynllrt00000gn/T/ipykernel_56668/1492736127.py:6: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  region_boot = region_boot.append(material_sum_revenue(target_subsample, probs_subsample, 200), ignore_index=True)
/var/folders/cy/1f9vz20x39s2ctm4tynllrt00000gn/T/ipykernel_56668/1492736127.py:6: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  region_boot = region_boot.append(material_sum_revenue(target_subsample, probs_subsample, 200), ignore_index=True)
/var/fo

/var/folders/cy/1f9vz20x39s2ctm4tynllrt00000gn/T/ipykernel_56668/1492736127.py:6: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  region_boot = region_boot.append(material_sum_revenue(target_subsample, probs_subsample, 200), ignore_index=True)
/var/folders/cy/1f9vz20x39s2ctm4tynllrt00000gn/T/ipykernel_56668/1492736127.py:6: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  region_boot = region_boot.append(material_sum_revenue(target_subsample, probs_subsample, 200), ignore_index=True)
/var/folders/cy/1f9vz20x39s2ctm4tynllrt00000gn/T/ipykernel_56668/1492736127.py:6: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  region_boot = region_boot.append(material_sum_revenue(target_subsample, probs_subsample, 200), ignore_index=True)
/var/fo

/var/folders/cy/1f9vz20x39s2ctm4tynllrt00000gn/T/ipykernel_56668/1492736127.py:6: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  region_boot = region_boot.append(material_sum_revenue(target_subsample, probs_subsample, 200), ignore_index=True)
/var/folders/cy/1f9vz20x39s2ctm4tynllrt00000gn/T/ipykernel_56668/1492736127.py:6: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  region_boot = region_boot.append(material_sum_revenue(target_subsample, probs_subsample, 200), ignore_index=True)
/var/folders/cy/1f9vz20x39s2ctm4tynllrt00000gn/T/ipykernel_56668/1492736127.py:6: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  region_boot = region_boot.append(material_sum_revenue(target_subsample, probs_subsample, 200), ignore_index=True)
/var/fo

/var/folders/cy/1f9vz20x39s2ctm4tynllrt00000gn/T/ipykernel_56668/1492736127.py:6: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  region_boot = region_boot.append(material_sum_revenue(target_subsample, probs_subsample, 200), ignore_index=True)
/var/folders/cy/1f9vz20x39s2ctm4tynllrt00000gn/T/ipykernel_56668/1492736127.py:6: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  region_boot = region_boot.append(material_sum_revenue(target_subsample, probs_subsample, 200), ignore_index=True)
/var/folders/cy/1f9vz20x39s2ctm4tynllrt00000gn/T/ipykernel_56668/1492736127.py:6: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  region_boot = region_boot.append(material_sum_revenue(target_subsample, probs_subsample, 200), ignore_index=True)
/var/fo

...

/var/folders/cy/1f9vz20x39s2ctm4tynllrt00000gn/T/ipykernel_56668/1492736127.py:6: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  region_boot = region_boot.append(material_sum_revenue(target_subsample, probs_subsample, 200), ignore_index=True)
/var/folders/cy/1f9vz20x39s2ctm4tynllrt00000gn/T/ipykernel_56668/1492736127.py:6: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  region_boot = region_boot.append(material_sum_revenue(target_subsample, probs_subsample, 200), ignore_index=True)
/var/folders/cy/1f9vz20x39s2ctm4tynllrt00000gn/T/ipykernel_56668/1492736127.py:6: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  region_boot = region_boot.append(material_sum_revenue(target_subsample, probs_subsample, 200), ignore_index=True)
/var/fo

/var/folders/cy/1f9vz20x39s2ctm4tynllrt00000gn/T/ipykernel_56668/1492736127.py:6: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  region_boot = region_boot.append(material_sum_revenue(target_subsample, probs_subsample, 200), ignore_index=True)
/var/folders/cy/1f9vz20x39s2ctm4tynllrt00000gn/T/ipykernel_56668/1492736127.py:6: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  region_boot = region_boot.append(material_sum_revenue(target_subsample, probs_subsample, 200), ignore_index=True)
/var/folders/cy/1f9vz20x39s2ctm4tynllrt00000gn/T/ipykernel_56668/1492736127.py:6: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  region_boot = region_boot.append(material_sum_revenue(target_subsample, probs_subsample, 200), ignore_index=True)
/var/fo

/var/folders/cy/1f9vz20x39s2ctm4tynllrt00000gn/T/ipykernel_56668/1492736127.py:6: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  region_boot = region_boot.append(material_sum_revenue(target_subsample, probs_subsample, 200), ignore_index=True)
/var/folders/cy/1f9vz20x39s2ctm4tynllrt00000gn/T/ipykernel_56668/1492736127.py:6: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  region_boot = region_boot.append(material_sum_revenue(target_subsample, probs_subsample, 200), ignore_index=True)
/var/folders/cy/1f9vz20x39s2ctm4tynllrt00000gn/T/ipykernel_56668/1492736127.py:6: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  region_boot = region_boot.append(material_sum_revenue(target_subsample, probs_subsample, 200), ignore_index=True)
/var/fo

/var/folders/cy/1f9vz20x39s2ctm4tynllrt00000gn/T/ipykernel_56668/1492736127.py:6: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  region_boot = region_boot.append(material_sum_revenue(target_subsample, probs_subsample, 200), ignore_index=True)
/var/folders/cy/1f9vz20x39s2ctm4tynllrt00000gn/T/ipykernel_56668/1492736127.py:6: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  region_boot = region_boot.append(material_sum_revenue(target_subsample, probs_subsample, 200), ignore_index=True)
/var/folders/cy/1f9vz20x39s2ctm4tynllrt00000gn/T/ipykernel_56668/1492736127.py:6: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  region_boot = region_boot.append(material_sum_revenue(target_subsample, probs_subsample, 200), ignore_index=True)
/var/fo

/var/folders/cy/1f9vz20x39s2ctm4tynllrt00000gn/T/ipykernel_56668/1492736127.py:6: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  region_boot = region_boot.append(material_sum_revenue(target_subsample, probs_subsample, 200), ignore_index=True)
/var/folders/cy/1f9vz20x39s2ctm4tynllrt00000gn/T/ipykernel_56668/1492736127.py:6: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  region_boot = region_boot.append(material_sum_revenue(target_subsample, probs_subsample, 200), ignore_index=True)
/var/folders/cy/1f9vz20x39s2ctm4tynllrt00000gn/T/ipykernel_56668/1492736127.py:6: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  region_boot = region_boot.append(material_sum_revenue(target_subsample, probs_subsample, 200), ignore_index=True)
/var/fo

In [27]:
print(region_0_boot,region_1_boot,region_2_boot)

     material_sum  material_revenue
0    23053.381323      3.740216e+08
1    22490.609816      1.207744e+08
2    23020.577552      3.592599e+08
3    24113.285802      8.509786e+08
4    22870.239558      2.916078e+08
..            ...               ...
995  22367.835066      6.552578e+07
996  23748.508232      6.868287e+08
997  22092.965717     -5.816543e+07
998  22800.433689      2.601952e+08
999  23204.042091      4.418189e+08

[1000 rows x 2 columns]      material_sum  material_revenue
0    23486.795219      5.690578e+08
1    22701.971547      2.158872e+08
2    23031.768884      3.642960e+08
3    22519.656925      1.338456e+08
4    23370.965308      5.169344e+08
..            ...               ...
995  23875.198622      7.438394e+08
996  22594.158503      1.673713e+08
997  23800.697044      7.103137e+08
998  22965.145952      3.343157e+08
999  23673.809384      6.532142e+08

[1000 rows x 2 columns]      material_sum  material_revenue
0    23491.615147      5.712268e+08
1    24025.279

Let's calculate the confidence intervals by declaring a function:

In [28]:
def get_confidence_interval(region_boot):
    return (region_boot.material_revenue.quantile(0.025), region_boot.material_revenue.quantile(0.975))

In [29]:
confidence_interval_region_0 = get_confidence_interval(region_0_boot)
confidence_interval_region_1 = get_confidence_interval(region_1_boot)
confidence_interval_region_2 = get_confidence_interval(region_2_boot)

Provicial risk evaluation

Let's declare a function called `risk_coeff` that returns the risk coefficient in percentage:

In [30]:
def risk_coeff(region_boot):
    low_treshold_count = len(region_boot.query('material_revenue <= 0'))
    return low_treshold_count/len(region_boot) * 100

In [31]:
risk_region_0 = risk_coeff(region_0_boot)
risk_region_1 = risk_coeff(region_1_boot)
risk_region_2 = risk_coeff(region_2_boot)

Let's create a final table that assesses the risk of investments made in the development of each province:

In [32]:
region_final = pd.DataFrame({'Avg provicial profit, RUB.'      : [region_0_boot.material_revenue.mean(),
                                                                     region_1_boot.material_revenue.mean(),
                                                                     region_2_boot.material_revenue.mean()],
                             '95% conf. interval, RUB.' : [confidence_interval_region_0,
                                                                     confidence_interval_region_1,
                                                                     confidence_interval_region_2],
                             'Probability of Losses, %'             : [risk_region_0,
                                                                     risk_region_1,
                                                                     risk_region_2]}, index=['Province 1','Province 2','Province 3'])
region_final

,"Avg provicial profit, RUB.","95% conf. interval, RUB.","Probability of Losses, %"
Province 1,4.319525e+08,"(-107449192.81585954, 957176650.0352675)",5.7
Province 2,5.161004e+08,"(75226938.32674514, 938818751.7522503)",1.2
Province 3,4.325050e+08,"(-132458348.03438877, 980238115.2749733)",5.6


## Conclusion

Based on the results of the project, we can recommend proceeding with the development of the third region(province). Despite having the maximum explored reserves, the model demonstrated a minimal risk coefficient. However, it should be noted that the average profit in the second region was higher, albeit with a higher risk coefficient. It would be prudent to request additional expert assessment of the provinces by geologists.